In [2]:
import pandas as pd

In [3]:
food_data = pd.read_csv('foodDesertLocator2006.csv')

In [5]:
# fix leading zeros in 'TRACT_FIPS' and ensure it's a string
food_data['Tract_FIPS'] = food_data['Tract_FIPS'].astype(str).str.zfill(11)

In [6]:
# extract the 5-digit County FIPS from the 11-digit Tract FIPS
food_data['county_fips'] = food_data['Tract_FIPS'].str[:5]

In [12]:
# compute weighted averages for each county
county_food_data = (
    food_data
    .groupby(['county_fips', 'CountyName', 'State'])  # groupby the extracted County FIPS
    .apply(
        lambda x: pd.Series({
            'total_population': x['TOTALPOP'].sum(),
            'weighted_low_access': (x['TOTALPOP'] * x['PERCENT_LOWA_Pop']).sum() / x['TOTALPOP'].sum(),
            'weighted_low_income_access': (x['TOTALPOP'] * x['PERCENT_LOWI']).sum() / x['TOTALPOP'].sum()
        })
    )
    .reset_index()  # reset index to make 'county_fips' a column again
)

In [13]:
county_food_data.head()

,county_fips,CountyName,State,total_population,weighted_low_access,weighted_low_income_access
0,01001,Autauga,AL,4874.0,53.260812,11.651908
1,01003,Baldwin,AL,3724.0,44.200000,10.300000
2,01005,Barbour,AL,12959.0,21.931206,6.238730
3,01011,Bullock,AL,3523.0,61.961993,20.863497
4,01013,Butler,AL,1693.0,42.700000,10.900000


In [14]:
# round the weighted averages to 2 decimal places
county_food_data['weighted_low_access'] = county_food_data['weighted_low_access'].round(2)
county_food_data['weighted_low_income_access'] = county_food_data['weighted_low_income_access'].round(2)

In [15]:
county_food_data.head()

,county_fips,CountyName,State,total_population,weighted_low_access,weighted_low_income_access
0,01001,Autauga,AL,4874.0,53.26,11.65
1,01003,Baldwin,AL,3724.0,44.20,10.30
2,01005,Barbour,AL,12959.0,21.93,6.24
3,01011,Bullock,AL,3523.0,61.96,20.86
4,01013,Butler,AL,1693.0,42.70,10.90


In [16]:
county_food_data.shape

(1851, 6)

In [17]:
# Save the processed county-level dataset for validation or merging
county_food_data.to_csv('county_food_data.csv', index=False)